In [19]:
import os
from pathlib import Path
os.chdir(Path("D:/Sagar_office/repo/chicken-disease-classification-Project"))

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_pramas : dict
    params_image_size : list
    params_batch_size : int

In [21]:
# cofiguration manager
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

# config manager
class ConfigurationManager:
     
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = Path("artifacts/training/model.h5"),
            training_data = Path("artifacts/data_ingestion/Chicken-fecal-images"),
            all_pramas = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

In [ ]:
from urllib.parse import urlparse
import tensorflow as tf
from src.cnnClassifier.utils.common import save_json

model = tf.keras.models.load_model("artifacts/training/model.h5")

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

In [24]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
   raise e


valid_generator = valid_datagenerator.flow_from_directory(
            directory=Path("artifacts/data_ingestion/Chicken-fecal-images"),
            subset="validation",
            shuffle=False
            )

[2024-11-01 11:20:00,213: INFO: yaml file: config\config.yaml loaded successfully]
[2024-11-01 11:20:00,251: INFO: yaml file: params.yaml loaded successfully]
[2024-11-01 11:20:00,255: INFO: create directory at: artifacts]
Found 116 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.